In [10]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 05 10:51:35 2020

@author: dell
"""

import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

def Find_Threshold(delta):#OTSU寻找阈值
    # 求灰度方差最大的那个数
    val=np.zeros([256])
    for th in range(256):
        loc1=delta>th
        loc2=delta<=th
        '''delta[loc1]=255
        delta[loc2]=0'''
        if delta[loc1].size==0:
            mu1=0
            omega1=0
        else:
            mu1=np.mean(delta[loc1])
            omega1=delta[loc1].size/delta.size

        if delta[loc2].size==0:
            mu2=0
            omega2=0
        else:
            mu2=np.mean(delta[loc2])
            omega2=delta[loc2].size/delta.size
        val[th]=omega1*omega2*np.power((mu1-mu2),2)

    #print("val=",val.shape)
    plt.figure()
    loc=np.where(val==np.max(val))
    #x=np.arange(0,256,1)
    #x=x.reshape([1,256])
    plt.plot(val)
    plt.ylabel("Var")
    plt.xlabel("Threshold")
    plt.grid("on")

    print("\nThe best OTSU Threshold: ",loc[0])
    return loc[0]

# 代码实现2w+1 w采用的是1 
def rcva(img1,img2): 
    b,g,red=cv2.split(img1)
    b1,g1,red1=cv2.split(img2)

    (row,cloumn)=b.shape
    #上下左右 左上 右上 左下 右下的坐标变化
    x=[-1,1,0,0,-1,-1,1,1]
    y=[0,0,-1,1,-1,1,-1,1]
    
    #保存变化后的差异图
    img_a=np.zeros((row,cloumn))
    img_b=np.zeros((row,cloumn))
    print("正在计算")
    for i in range(row):
        for j in range(cloumn):
            if i!=0 and j!=0 and i!=row and j!=cloumn:
                r=i
                c=j
                res1=1000000.0
                res2=1000000.0
                res3=1000000.0
                for k in range(8):
                    if(0<r and r<row and c>0 and c<cloumn):
                        kk1=(math.pow(abs(int(b[i,j])-int(b1[r,c])),2))
                        if(kk1<res1):
                            res1=kk1
                        kk2=(math.pow(abs(int(g[i,j])-int(g1[r,c])),2))
                        if(kk2<res2):
                            res2=kk2
                        kk3=(math.pow(abs(int(red[i,j])-int(red1[r,c])),2))
                        if(kk3<res3):
                            res3=kk3
                    r=i
                    c=j
                    r+=x[k]
                    c+=y[k]
                img_a[i][j]=math.sqrt(res1+res2+res3)
                
        # 考虑四个边角 和四个边界
            else:
                img_a[i,j]=abs(int(b[i,j])-int(b1[i,j]))
    
    print("正在计算中请稍等")
    
    for i in range(row):
        for j in range(cloumn):
            if i!=0 and j!=0 and i!=row and j!=cloumn:
                r=i
                c=j
                res1=1000000.0
                res2=1000000.0
                res3=1000000.0
                for k in range(8):
                    if(0<r and r<row and c>0 and c<cloumn):
                        kk1=(math.pow(abs(int(b[i,j])-int(b1[r,c])),2))
                        if(kk1<res1):
                            res1=kk1
                        kk2=(math.pow(abs(int(g[i,j])-int(g1[r,c])),2))
                        if(kk2<res2):
                            res2=kk2
                        kk3=(math.pow(abs(int(red[i,j])-int(red1[r,c])),2))
                        if(kk3<res3):
                            res3=kk3
                    r=i
                    c=j
                    r+=x[k]
                    c+=y[k]
                img_b[i][j]=math.sqrt(res1+res2+res3)
        # 考虑四个边角 和四个边界
            else:
                img_b[i,j]=abs(int(b1[i,j])-int(b[i,j]))

    img_b_change=np.zeros((row,cloumn))
    for i in range(row):
        for j in range(cloumn):
            if img_a[i,j]>img_b[i,j]:
                img_b_change[i,j]=img_b[i,j]
            else:
                img_b_change[i,j]=img_a[i,j]

    print("计算完成正在二值化")
    th=Find_Threshold(img_b_change)
    
    for i1 in range(row):
        for i2 in range(cloumn):
            if img_b_change[i1][i2]<=th:
                img_b_change[i1][i2]=0
            else:
                img_b_change[i1][i2]=255
    cv2.imshow('diff3',img_b_change)
    cv2.imwrite("..\RXX-2020-CVA\Testing dataset\B-landslideAerialImage/rcva.bmp",img_b_change)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def cva(img1,img2):
    b,g,r=cv2.split(img1)
    b1,g1,r1=cv2.split(img2)

    (row,col)=b1.shape

    d_1=cv2.subtract(b,b1)
    
    d_2=cv2.subtract(g,g1)
    d_3=cv2.subtract(r,r1)

    d_1=cv2.pow(d_1,2)
    d_2=cv2.pow(d_2,2)
    d_3=cv2.pow(d_3,2)

    d_sum_2=np.add(d_1,d_2,d_3)
    #d_sum_2 = cv2.cvtColor(d_sum_2, cv2.COLOR_BGR2GRAY)
    
    for i in range(row):
        for j in range(col):
            d_sum_2[i,j]=math.sqrt(d_sum_2[i,j])
    delta=d_sum_2
    
    
    th=Find_Threshold(delta)
   
    if np.size(th)>1:
        th=th[0]
    for i1 in range(row):
        for i2 in range(col):
            if delta[i1][i2]>=th:
                delta[i1][i2]=255
            else:
                delta[i1][i2]=0
    cv2.imshow('diff3',delta)
    cv2.imwrite("..\RXX-2020-CVA\Testing dataset\TianJing-SPOT/rcva.bmp",delta)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

img1=cv2.imread(r'C:\Users\dell\Desktop\RXX-2020-CVA\RXX-2020-CVA\Testing dataset\TianJing-SPOT\09BMP.bmp')
img2=cv2.imread(r'C:\Users\dell\Desktop\RXX-2020-CVA\RXX-2020-CVA\Testing dataset\B-landslideAerialImage/2014dopdyssubB_CopyRaster.tif')
print(img1.shape)
#rcva(img1,img2)



(451, 484, 3)
